## Эксперименты с моделями

### Импорт зависимостей

**Загрузка библиотек**

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [29]:
import numpy as np

In [26]:
from sklearn.naive_bayes import GaussianNB

**Зарузка данных**

In [8]:
df = pd.read_parquet('df.parquet')  # Формируется в ноутбуке preprocessing
X = df.cleaned_tweet  # Еще нужно достать оттуда какое-то векторное представление
y = df.feeling  # Куда делся df.lemmatized??

### Простые правила

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
y_pred = np.random.randint(low=0, high=1, size=y_test.shape[0])
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 74993 points : 37489


### Простые модели, BoW

**Preprocessing**

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features=100)
X = tfidf_vectorizer.fit_transform(X).toarray()

**BoW + Наивный Байес**

[Описание](https://scikit-learn.org/1.5/modules/naive_bayes.html)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 74993 points : 26428


**BoW + $k$ ближайших соседей**

[Описание](https://scikit-learn.org/1.5/modules/naive_bayes.html)

**BoW + Логистическая регрессия**

[Описание](https://scikit-learn.org/1.5/modules/naive_bayes.html)